In [43]:
import pandas as pd
import numpy as np
data = pd.read_csv('emscad_v1.csv')

In [44]:
data = data.drop('salary_range', axis=1)

In [45]:
# drop duplicates
data.drop_duplicates(inplace=True)
data.duplicated().sum()


0

In [46]:
columns_to_convert = ['telecommuting', 'has_company_logo', 'has_questions', 'fraudulent', 'in_balanced_dataset']
for col in columns_to_convert:
    data[col] = data[col].replace({'t': 1, 'f': 0}).astype(int)
data['employment_type'] = data['employment_type'].replace(np.nan, 'Other')
data['required_experience'] = data['required_experience'].replace(np.nan, 'Not Applicable')
data['required_education'] = data['required_education'].replace(np.nan, 'Unspecified')
data['industry'] = data['industry'].replace(np.nan, 'Other')
data['function'] = data['function'].replace(np.nan, 'Other')

In [47]:
data['missing location'] = data['location'].apply(lambda x: 1 if pd.isna(x) else 0)
data['missing department'] = data['department'].apply(lambda x: 1 if pd.isna(x) else 0)
data['missing company profile'] = data['company_profile'].apply(lambda x: 1 if pd.isna(x) else 0)
data['missing requirements'] = data['requirements'].apply(lambda x: 1 if pd.isna(x) else 0)
data['missing benefits'] = data['benefits'].apply(lambda x: 1 if pd.isna(x) else 0)

In [48]:
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('sentiwordnet')
# from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import re
from bs4 import BeautifulSoup
from sklearn.preprocessing import LabelEncoder




[nltk_data] Downloading package stopwords to /Users/yaoyi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/yaoyi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [49]:
def remove_html_tags(text):
    text = str(text)
    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(text, 'html.parser')
    
    # Extract the text content without HTML tags
    text_without_tags = soup.get_text()
    
    # Remove any extra whitespace and newline characters
    text_without_tags = re.sub(r'\s+', ' ', text_without_tags).strip()
    
    return text_without_tags


In [50]:
# replace US with USA
data['location'] = data['location'].str.replace('US', 'USA')

In [51]:
columns_to_count = ['title', 'location', 'department']
for col in columns_to_count:
    data[col] = data[col].replace(np.nan, '')
    data[str.format('{}_length', col)] = data[col].apply(len)

In [52]:
columns_to_count = ['company_profile', 'description', 'requirements']
for col in columns_to_count:
    data[col] = data[col].replace(np.nan, '')
    data[col] = data[col].apply(remove_html_tags)
    data[str.format('{}_length', col)] = data[col].apply(len)

/Users/yaoyi/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [53]:
data

,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,...,missing department,missing company profile,missing requirements,missing benefits,title_length,location_length,department_length,company_profile_length,description_length,requirements_length
0,Marketing Intern,"USA, NY, New York",Marketing,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,...,0,0,0,1,16,17,9,888,908,862
1,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you: Your key responsibili...,<h3><b>What you will get from us</b></h3>\r\n<...,0,1,0,...,0,0,0,0,41,14,7,1289,2072,1439
2,Commissioning Machinery Assistant (CMA),"USA, IA, Wever",,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,...,1,0,0,1,39,14,0,875,355,1377
3,Account Executive - Washington DC,"USA, DC, Washington",Sales,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",<p>Our culture is anything but corporate—we ha...,0,1,0,...,0,0,0,0,33,19,5,613,2617,1444
4,Bill Review Manager,"USA, FL, Fort Worth",,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review Manager LOCATION...,QUALIFICATIONS: RN license in the State of Tex...,<p>Full Benefits Offered</p>,0,1,1,...,1,0,0,0,19,19,0,1632,1445,769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,Account Director - Distribution,"CA, ON, Toronto",Sales,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you: Will eat comprehensive S...,<p><b>What can you expect from us?</b></p>\r\n...,0,1,1,...,0,0,0,0,32,15,5,1715,1437,1271
17876,Payroll Accountant,"USA, PA, Philadelphia",Accounting,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting - Desire to have ...,<p></p>\r\n<h3>Health &amp; Wellness</h3>\r\n<...,0,1,1,...,0,0,0,0,18,21,10,2273,1217,728
17877,Project Cost Control Staff Engineer - Cost Con...,"USA, TX, Houston",,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience. Abi...,NaN,0,0,0,...,1,0,0,1,59,16,0,228,1301,1240
17878,Graphic Designer,"NG, LA, Lagos",,,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,<p>Competitive salary (compensation will be ba...,0,0,1,...,1,1,0,0,16,13,0,0,506,494


In [54]:
def preprocess(text):
    text = remove_html_tags(text)
    text = re.sub(pattern='[^a-zA-Z0-9]',repl=' ',string=text)
    text = text.lower()
    tokens = word_tokenize(text)
    stop = set(stopwords.words('english'))
    tokens = [str(word) for word in tokens if word not in stop]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # ps = PorterStemmer()
    # tokens = [ps.stem(word) for word in tokens]
    sentence = ' '.join(tokens)
    return sentence

In [55]:
data['title'] = data['title'].apply(preprocess)
data['location'] = data['location'].apply(preprocess)
data['department'] = data['department'].apply(preprocess)
data['company_profile'] = data['company_profile'].apply(preprocess)
data['description'] = data['description'].apply(preprocess)
data['requirements'] = data['requirements'].apply(preprocess)
data['benefits'] = data['benefits'].apply(preprocess)
data['industry'] = data['industry'].apply(preprocess)
data['function'] = data['function'].apply(preprocess)

/Users/yaoyi/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [56]:
label_encoder = LabelEncoder()
columns_to_encode = ['employment_type', 'required_experience', 'required_education']
for col in columns_to_encode:
    data[col] = label_encoder.fit_transform(data[col])
    labels = label_encoder.classes_
    print(col)
    for i in range(len(labels)):
        print(f'{i}: {labels[i]}')

employment_type
0: Contract
1: Full-time
2: Other
3: Part-time
4: Temporary
required_experience
0: Associate
1: Director
2: Entry level
3: Executive
4: Internship
5: Mid-Senior level
6: Not Applicable
required_education
0: Associate Degree
1: Bachelor's Degree
2: Certification
3: Doctorate
4: High School or equivalent
5: Master's Degree
6: Professional
7: Some College Coursework Completed
8: Some High School Coursework
9: Unspecified
10: Vocational
11: Vocational - Degree
12: Vocational - HS Diploma


In [57]:
data['title'] = data['title'].replace('nan', '')
data['location'] = data['location'].replace('nan', '')
data['department'] = data['department'].replace('nan', '')
data['company_profile'] = data['company_profile'].replace('nan', '')
data['description'] = data['description'].replace('nan', '')
data['requirements'] = data['requirements'].replace('nan', '')
data['benefits'] = data['benefits'].replace('nan', '')
data['industry'] = data['industry'].replace('nan', '')
data['function'] = data['function'].replace('nan', '')

In [58]:
data['full_text_length'] = data['title_length'] + data['location_length'] + data['department_length'] + data['company_profile_length'] + data['description_length'] + data['requirements_length']

In [59]:
data


,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,...,missing company profile,missing requirements,missing benefits,title_length,location_length,department_length,company_profile_length,description_length,requirements_length,full_text_length
0,marketing intern,usa ny new york,marketing,food52 created groundbreaking award winning co...,food52 fast growing james beard award winning ...,experience content management system major plu...,,0,1,0,...,0,0,1,16,17,9,888,908,862,2700
1,customer service cloud video production,nz auckland,success,90 second world cloud video production service...,organised focused vibrant awesome passion cust...,expect key responsibility communicate client 9...,get u part 90 second team gain experience work...,0,1,0,...,0,0,0,41,14,7,1289,2072,1439,4862
2,commissioning machinery assistant cma,usa ia wever,,valor service provides workforce solution meet...,client located houston actively seeking experi...,implement pre commissioning commissioning proc...,,0,1,0,...,0,0,1,39,14,0,875,355,1377,2660
3,account executive washington dc,usa dc washington,sale,passion improving quality life geography heart...,company esri environmental system research ins...,education bachelor master gi business administ...,culture anything corporate collaborative creat...,0,1,0,...,0,0,0,33,19,5,613,2617,1444,4731
4,bill review manager,usa fl fort worth,,spotsource solution llc global human capital m...,job title itemization review manager location ...,qualification rn license state texas diploma b...,full benefit offered,0,1,1,...,0,0,0,19,19,0,1632,1445,769,3884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,account director distribution,ca toronto,sale,vend looking awesome new talent come join u wo...,case first time visited website vend award win...,ace role eat comprehensive statement work brea...,expect u open culture openly share result inpu...,0,1,1,...,0,0,0,32,15,5,1715,1437,1271,4475
17876,payroll accountant,usa pa philadelphia,accounting,weblinc e commerce platform service provider f...,payroll accountant focus primarily payroll fun...,b b accounting desire fun love genuine passion...,health wellness medical plan prescription drug...,0,1,1,...,0,0,0,18,21,10,2273,1217,728,4267
17877,project cost control staff engineer cost contr...,usa tx houston,,provide full time permanent position many medi...,experienced project cost control staff enginee...,least 12 year professional experience ability ...,,0,0,0,...,0,0,1,59,16,0,228,1301,1240,2844
17878,graphic designer,ng la lagos,,,nemsia studio looking experienced visual graph...,1 must fluent latest version corel adobe cc es...,competitive salary compensation based experien...,0,0,1,...,1,0,0,16,13,0,0,506,494,1029


In [60]:
data['missing company profile'] = data['company_profile'].apply(lambda x: 1 if (len(x) == 0) else 0)

In [61]:
from sklearn.preprocessing import LabelEncoder

# Create a new column for missing values combination
data['missing_combination'] = ''

# Define conditions for each combination
mask_both_present = (data['has_company_logo'] == 1) & (data['missing company profile'] == 0)
mask_missing_logo = (data['has_company_logo'] == 0) & (data['missing company profile'] == 0)
mask_missing_profile = (data['has_company_logo'] == 1) & (data['missing company profile'] == 1)
mask_both_missing = (data['has_company_logo'] == 0) & (data['missing company profile'] == 1)

# Assign labels based on conditions
data.loc[mask_both_present, 'missing_combination'] = 'Both profile and logo Present'
data.loc[mask_missing_logo, 'missing_combination'] = 'Missing Logo Only'
data.loc[mask_missing_profile, 'missing_combination'] = 'Missing Profile Only'
data.loc[mask_both_missing, 'missing_combination'] = 'Both profile and logo Missing'

# Use one-hot encoding
one_hot_encoded = pd.get_dummies(data['missing_combination'], prefix='missing_combination')


# Convert boolean values to 0 and 1
one_hot_encoded = one_hot_encoded.astype(int)

# Concatenate the one-hot encoded columns with the original dataset
data = pd.concat([data, one_hot_encoded], axis=1)

# Display the resulting DataFrame
print(data[['missing_combination'] + list(one_hot_encoded.columns)])
data

                 missing_combination  \
0      Both profile and logo Present   
1      Both profile and logo Present   
2      Both profile and logo Present   
3      Both profile and logo Present   
4      Both profile and logo Present   
...                              ...   
17875  Both profile and logo Present   
17876  Both profile and logo Present   
17877              Missing Logo Only   
17878  Both profile and logo Missing   
17879  Both profile and logo Present   

       missing_combination_Both profile and logo Missing  \
0                                                      0   
1                                                      0   
2                                                      0   
3                                                      0   
4                                                      0   
...                                                  ...   
17875                                                  0   
17876                                  

,title,location,department,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,...,department_length,company_profile_length,description_length,requirements_length,full_text_length,missing_combination,missing_combination_Both profile and logo Missing,missing_combination_Both profile and logo Present,missing_combination_Missing Logo Only,missing_combination_Missing Profile Only
0,marketing intern,usa ny new york,marketing,food52 created groundbreaking award winning co...,food52 fast growing james beard award winning ...,experience content management system major plu...,,0,1,0,...,9,888,908,862,2700,Both profile and logo Present,0,1,0,0
1,customer service cloud video production,nz auckland,success,90 second world cloud video production service...,organised focused vibrant awesome passion cust...,expect key responsibility communicate client 9...,get u part 90 second team gain experience work...,0,1,0,...,7,1289,2072,1439,4862,Both profile and logo Present,0,1,0,0
2,commissioning machinery assistant cma,usa ia wever,,valor service provides workforce solution meet...,client located houston actively seeking experi...,implement pre commissioning commissioning proc...,,0,1,0,...,0,875,355,1377,2660,Both profile and logo Present,0,1,0,0
3,account executive washington dc,usa dc washington,sale,passion improving quality life geography heart...,company esri environmental system research ins...,education bachelor master gi business administ...,culture anything corporate collaborative creat...,0,1,0,...,5,613,2617,1444,4731,Both profile and logo Present,0,1,0,0
4,bill review manager,usa fl fort worth,,spotsource solution llc global human capital m...,job title itemization review manager location ...,qualification rn license state texas diploma b...,full benefit offered,0,1,1,...,0,1632,1445,769,3884,Both profile and logo Present,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,account director distribution,ca toronto,sale,vend looking awesome new talent come join u wo...,case first time visited website vend award win...,ace role eat comprehensive statement work brea...,expect u open culture openly share result inpu...,0,1,1,...,5,1715,1437,1271,4475,Both profile and logo Present,0,1,0,0
17876,payroll accountant,usa pa philadelphia,accounting,weblinc e commerce platform service provider f...,payroll accountant focus primarily payroll fun...,b b accounting desire fun love genuine passion...,health wellness medical plan prescription drug...,0,1,1,...,10,2273,1217,728,4267,Both profile and logo Present,0,1,0,0
17877,project cost control staff engineer cost contr...,usa tx houston,,provide full time permanent position many medi...,experienced project cost control staff enginee...,least 12 year professional experience ability ...,,0,0,0,...,0,228,1301,1240,2844,Missing Logo Only,0,0,1,0
17878,graphic designer,ng la lagos,,,nemsia studio looking experienced visual graph...,1 must fluent latest version corel adobe cc es...,competitive salary compensation based experien...,0,0,1,...,0,0,506,494,1029,Both profile and logo Missing,1,0,0,0


In [62]:
data.to_csv('preprocessed_data_with_num.csv', index=False)

In [63]:
data.columns

Index(['title', 'location', 'department', 'company_profile', 'description',
       'requirements', 'benefits', 'telecommuting', 'has_company_logo',
       'has_questions', 'employment_type', 'required_experience',
       'required_education', 'industry', 'function', 'fraudulent',
       'in_balanced_dataset', 'missing location', 'missing department',
       'missing company profile', 'missing requirements', 'missing benefits',
       'title_length', 'location_length', 'department_length',
       'company_profile_length', 'description_length', 'requirements_length',
       'full_text_length', 'missing_combination',
       'missing_combination_Both profile and logo Missing',
       'missing_combination_Both profile and logo Present',
       'missing_combination_Missing Logo Only',
       'missing_combination_Missing Profile Only'],
      dtype='object')